# Labo 7 : classification de textes (pour la désambiguïsation lexicale)
Nathan Gonzalez Montes & Guidoux Vincent

## Objectif et plan

L’objectif de ce laboratoire est d’utiliser des méthodes d’apprentissage supervisé pour classifier des occurrences du mot interest selon leur sens : c’est la même tâche, avec les mêmes données, que le labo 6. Pour classifier les occurrences, on considère leur contexte (mots voisins), et on applique l’approche bayésienne vue en cours : on entraîne un classifieur à six classes (les six sens de interest annotés de 1 à 6) sur une partie des données et on le teste sur la partie restante.

Dans ce laboratoire, on explore deux façons différentes de coder les traits (features) pour cette tâche. Dans les deux cas, on entraînera un NaiveBayesClassifier fourni par NLTK.1 Les deux façons sont :
1. Constituer un vocabulaire des mots qui apparaissent dans le voisinage de interest et définir ces mots comme traits. Pour chaque occurrence de interest, on extrait la valeur de ces traits sous la forme `{(‘rate’ : True), (‘in’ : False), … }` et on ajoute la classe (de 1 à 6).
2. Si word-1 est le mot précédant l’occurrence de interest, on définit comme traits word-n, …, word-2, word-1, word+1, word+2, …, word+n (une fenêtre de taille 2n autour de interest). Les valeurs possibles de ces traits sont cette fois-ci les mots observés, ou ‘NONE’ si la fenêtre dépasse les limites de la phrase. Pour chaque occurrence de interest, on extrait la valeur de ces traits sous la forme `{(‘word-1’ : ‘his’), (‘word+1’ : ‘in’), … }` et on ajoute la classe (de 1 à 6).

Dans les deux cas, il faut diviser les 2368 occurrences de interest en un jeu d’entraînement et un jeu de test, en respectant la proportion initiale de chaque sens. Puis on entraîne un classifieur bayésien naïf en respectant le format de données indiqué par NLTK2, et on teste la performance du classifieur entraîné. L’objectif est de trouver les paramètres qui conduisent aux meilleurs scores de WSD.

## Importation

In [1]:
import nltk.tokenize

## Liste des noms propres, ponctuation :

In [2]:
stopWords_dlc = ["'", "''", "'s", '(', ')', '*interest', '*interests', ',', ':', ';', 'At least one of the passed list is empty',
 'USsr','``', 'addwest', 'allianz', 'amoco', 'angell', 'angevine', 'anheuser', 'artra', 'avedisian', 'bancorp',
 'bankamerica', 'bankshares', 'bavaria', 'beghin', 'bentsen', 'bernhard', 'berson', 'beyer', 'birnbaum', 'bnl', 'bolduc', 'bsn',
 'buckley', 'bundesbank', 'caldor',  'campeau',  'canfor',  'cathay',  'cirino',  'citic',  'citicorp',  'citiesabc', 'comsat', 'conagra', 'curragh', 'daimler',
 'datatimes', 'dingell', 'doman', 'dougherty', 'drexel','drg', 'ecpa', 'eiji', 'elsinore', 'erc', 'ericson', 'ernst', 'esop', 'esso',
 'eubank', 'fasb', 'fii', 'geoffrey', 'giorgio', 'goloven', 'goodson', 'gorbachev', 'gosbank', 'grafil', 'harcourt', 'harriet',
 'hca', 'healthvest', 'heyden', 'hirsch', 'honeywell', 'hornbeck', 'icahn', 'interlake', 'itel', 'itoh', 'jerrold', 'kalmus',
 'kampen','kaneb', 'kangyo', 'kenyon', 'keogh', 'kerkorian', 'kikkoman', 'kissinger', 'klauser', 'kochan', 'krause', 'laidlaw',
 'lazar', 'lbo', 'lbos', 'leaseway', 'leber', 'lipper', 'lipsky', 'lipstein', 'lomas', 'lpl', 'makro', 'malizia', 'massey',
 'matra', 'matuschka', 'maxus', 'mccall', 'mccaw', 'mclennan', 'mcorp', 'meritor', 'metromedia', 'mikulich', 'milacron', 'milford', 'milken',
 'minorco', 'mitsui', 'mixte', 'mulford', 'nasd', 'ncnb', 'negus', 'nepalese', 'newfoundland', 'nippon', 'nobrega', 'northrop',
 'nrm','nyu', "o'malley", 'oakes', 'osborne', 'painewebber', 'panamanian', 'pantera', 'papandreou', 'parkos', 'paxus', 'pnc',
 'poehl', 'quotron', 'racal', 'rapanelli', 'raytheon', 'refcorp', 'revco', 'richfield', 'rjr', 'rockwell', 'rostenkowski',
 'rtc', 'ruiz', 'saatchi', 'sallie', 'salomon', 'sandner', 'satoshi', 'schlossberg', 'schlumberger', 'searby', 'seife', 'shapiro',
 'silbert', 'sirrine', 'skase', 'smithkline', 'sotheby', 'sperandeo', 'spiotto', 'steinberg', 'swissair', 'sybase', 'takamori',
 'takeshi', 'tci', 'telerate', 'textron', 'thermedics', 'tomsho', 'transco', 'trecker', 'tva', 'uic', 'unilever', 'unisys', 'unocal',
 'viacom', 'vickers', 'viyella', 'vuitton', 'waigel', 'weinroth', 'westinghouse', 'weyerhaeuser', 'wisner', 'wyse', 'yeres', 'younkers',
 'yukio', 'zurich', '}', '\'why', '*interested', 'At least one of the passed list is empty', 'MGMNP', '\\\\*', 'aganbegyan', 'aloys',
 'alpert','altimari', 'altman', 'ameritas', 'anctil', 'ankeny', 'anlage', 'arby', 'asarco', 'atb', 'atwood', 'avdel', 'aviva',
 'balfour', 'bateman','bddp', 'berardi', 'berbera', 'bianchi', 'biondi', 'birkel', 'blandings', 'bloomingdale', 'bloomingdales',
 'blumenfeld', 'bodner','bolar', 'borden', 'boskin', 'braitman', 'bramah', 'brasilia', 'briscoe', 'burbank', 'burnham', 'burrill',
 'bynoe', 'cafferarelli', 'cambria', 'capel', 'cashin', 'chandross', 'ciminero', 'comerica', 'congolese', 'conlon', 'corbehem',
 'cowles', 'crandall', 'crss', 'daberko', 'daiwa', 'darman', 'depcreciation', 'dimitris', 'dlj', 'donaldson', 'donoghue', 'dorrance', 'doughnut',
 'dresdner', 'dreyer', 'edelson', 'elbaum', 'etr', 'farmington', 'faulding', 'fedders', 'ferranti', 'fio', 'firstsouth',
 'franyo', 'fulbright', 'fyffes', 'gaubert', 'gec', 'gintel', 'goodison', 'goodwin', 'gould', 'gramm', 'grubman', 'gutermann',
 'hammersmith', 'hanwa', 'hartzog', 'hawley', 'hbj', 'heilman', 'hennessey', 'hering', 'hibernia', 'holliston', 'howson',
 'hubbell', 'hyman', 'iberian', 'ikegai','intan', 'iras', 'itagaki', 'janson', 'jennison', 'jmb', 'jolla', 'jujo', 'kakita',
 'kansan', 'kenji', 'kirkland', 'kohlberg', 'kondo', 'koskotas', 'krauss', 'kuala', 'kulani', 'landini', 'lawrenceville',
 'lawrenson', 'lazard', 'lebow', 'lengwin', 'leningrad', 'leventhal', 'levine', 'levinson', 'levitt', 'lidgerwood', 'lightstone',
 'lintas', 'liro', 'lockheed', 'lorenzo', 'ludcke', 'lufthansa', 'lupo', 'lustgarten', 'lyphomed', 'mackenzie', 'maclean', 'malato',
 'mandle', 'matsuda', 'mcdermott','mcgraw', 'mckusick', 'mehl', 'menem', 'merksamer', 'merritt', 'messaggero', 'mgmua', 'minincomputer',
 'minpeco', 'mitsuoka', 'mnb', 'monieson', 'morningstar', 'murata', 'mvestment', 'nakazato', 'natwest', 'newsedge', 'nisshin',
 'nzi', "o'neill", 'olivetti', 'orgotein', 'ormat', 'packwood', 'pamour', 'parsow', 'pemberton', 'perlman', 'phelan', 'pocklington', 'pomicino', 'pountain',
 'prapas', 'prentice','primerit', 'qintex', 'quadrum', 'rainman', 'ralphs', 'recommendatons', 'redford', 'repsol', 'resler',
 'rodale', 'rosen', 'rothschild','roulac', 'ruderman', 'ruskin', 'sabre', 'sakowitz', 'salerno', 'sanyo', 'savaiko', 'schering',
 'schumer', 'scowcroft', 'seabrook', 'seiders', 'seidman', 'shearson', 'sheraton', 'shirer', 'southbrook', 'sovran', 'squibb',
 'stenholm', 'stevric', 'straszheim', 'sumita', 'superconcentrated', 'sykes', 'taft', 'telemunchen', 'terrizzi', 'tietmeyer',
 'trelleborg', 'tvsm', 'tvx', 'upham', 'vargas', 'viag', 'visx', 'wachtel', 'waterhouse', 'weissman', 'whitaker', 'whitehall',
 'whittaker', 'wyss', 'zoete', '{',  "'goodison", "'who", 'At least one of the passed list is empty', '\\\\*', 'algraphy',
 'alsthom', 'alun', 'anschluss', 'bache', 'baer', 'banxquote', 'bischofberger', 'caci', "d'amiante", 'economidis', 'eichler', 'eiffel','entergy',
 'eurobelge', 'exploracion', 'feldemuehle', 'ferruzzi', 'freres', 'galbraith', 'gmb', 'goldwyn', 'hutchinson', 'jaworski', 'jerrico',
 'jvcvictor', 'kokusai', 'kravis', 'leval', 'llosa', 'lufkin', 'lumpur', 'maclaine', 'maffei', 'matsushita', 'mca', 'moleculon',
 'neiman', 'normura', 'ocn', 'plough', 'proudfoot', 'puna', 'rudman', 'seger', 'sigoloff', 'sithe', 'stateswest', 'swasey','syncor',
 'synthetical', 'takashimaya', 'taunton','treausry', 'trupin', 'utsunomiya', 'warburg', 'wedd', 'wyo', 'yellowknife',
 "At least one of the passed list is empty", '\\\\*', 'bradstreet', 'economdis', 'forstmann', 'heileman', 'lorimar', 'luxembourg', 'pamorex',
 'saks', 'turben', 'wassily', '\\\\*', 'leontief', 'rauscher', 'telepictures', 'interest_', '.']

## Étapes proposées

### A. Traits lexicaux : présence ou absence de mots dans le voisinage de interest

#### 1. Le fichier de données se trouve à http://www.d.umn.edu/~tpederse/data.html – chercher « interest » vers la fin de la page, et prendre le fichier marqué comme « original format without POS tags » (le même qu’au labo 6). Lire le fichier et générer une liste de listes de mots (une liste par phrase) appelée `tokenized_sentences`.

In [3]:
filepath = 'data/interest-original.txt' 

try:  
    fp = open(filepath, 'r',encoding="utf-8")
    raw_sentences = fp.read().replace('=', '')
    raw_sentences = raw_sentences.replace(',', '')
    raw_sentences = raw_sentences.replace('.', '')
    raw_sentences = raw_sentences.replace('`', '')
    raw_sentences = raw_sentences.replace('\'', '')
    raw_sentences = raw_sentences.split('\n$$\n')
finally:  
    fp.close()
    
tokenized_sentences = [nltk.word_tokenize(sent) for sent in raw_sentences[:-1]]

len(tokenized_sentences)

2368

#### 2. Définir une variable `window_size`, par exemple égale à 3 (on la fera varier plus tard), et une liste vide de mots `word_list`. Parcourir les `tokenized_sentences` et pour chaque phrase ajouter les mots voisins de interest (i.e. situés à une distance inférieure ou égale à `window_size`) dans la liste de mots `word_list`. Combien de mots contient celle-ci à la fin ? Tokens ou types ?

In [4]:
windows_size = 3

word_dict = {1: [], 2: [], 3: [], 4: [], 5: [], 6: []}

for sentence in (tokenized_sentences):  
    i = None
    current_sens = None
    for def_i in range(1,7):
        current_sens = def_i
        if "interest_{}".format(def_i) in sentence:
            i = sentence.index("interest_{}".format(def_i))
            break
        elif "interests_{}".format(def_i) in sentence:
            i = sentence.index("interests_{}".format(def_i))
            break
    for index in range(-windows_size, windows_size+1):
        current_index = index + i
        if current_index > 0 and current_index < len(sentence) and index != 0:
            word_dict[current_sens].append(sentence[current_index])


In [5]:
somme = 0

vocabulary = set()
word_list = []

for i in range(1,7):
    number_of_point = 0
    current_list = word_dict[i]
    print("word_list {} : {}".format(i, len(current_list)))
    somme += len(current_list)
    for word in current_list:
        vocabulary.add(word)
        word_list.append(word)

word_list 1 : 1951
word_list 2 : 65
word_list 3 : 309
word_list 4 : 933
word_list 5 : 2773
word_list 6 : 6759


In [6]:
print("nombre total d'occurances : {}".format(somme))
print("nombre de mots différents : {}".format(len(vocabulary)))

nombre total d'occurances : 12790
nombre de mots différents : 2479


#### 3. À l’aide d’un objet `NLTK` de type `FreqDist`, sélectionner parmi les mots de `word_list` les `N` plus fréquents, dans une nouvelle liste appelée `vocabulary` (p.ex. `N = 500`, mais on le fera varier). Affichez les 50 mots les plus fréquents. Est-ce une bonne idée d’enlever les stopwords ?

In [7]:
N = 500
vocabulary = nltk.FreqDist(word_list).most_common(N)

In [8]:
nltk.FreqDist(word_list).most_common(50)

[('in', 762),
 ('rates', 624),
 ('the', 606),
 ('and', 401),
 ('to', 373),
 ('of', 330),
 ('a', 273),
 ('on', 170),
 ('rate', 139),
 ('s', 136),
 ('%', 135),
 ('its', 132),
 ('payments', 112),
 ('that', 104),
 ('are', 102),
 ('has', 94),
 ('for', 86),
 ('with', 83),
 ('lower', 83),
 ('an', 81),
 ('is', 72),
 ('by', 69),
 ('have', 68),
 ('will', 65),
 ('at', 64),
 ('high', 63),
 ('from', 61),
 ('company', 50),
 ('their', 48),
 ('or', 48),
 ('annual', 48),
 ('us', 47),
 ('which', 47),
 ('short', 47),
 ('minority', 46),
 ('foreign', 46),
 ('said', 43),
 ('higher', 43),
 ('as', 42),
 ('bonds', 42),
 ('income', 42),
 ('nt', 41),
 ('other', 41),
 ('it', 39),
 ('pay', 39),
 ('be', 35),
 ('below', 35),
 ('up', 34),
 ('would', 34),
 ('because', 33)]

**Est-ce une bonne idée d’enlever les stopwords ?**
> blabla

#### 4. Parcourir à nouveau les tokenized_sentences et pour chaque phrase créer un couple (dictionnaire, sens), où le dictionnaire regroupe les traits et leurs valeurs, et le sens est un nombre de 1 à 6 indiquant le sens de interest. Les couples pour toutes les phrases seront rassemblés dans une liste appelée feature_sets.

In [9]:
dictonary = {}

for trait in vocabulary:
    dictonary[trait[0]] = False
dictonary

{'in': False,
 'rates': False,
 'the': False,
 'and': False,
 'to': False,
 'of': False,
 'a': False,
 'on': False,
 'rate': False,
 's': False,
 '%': False,
 'its': False,
 'payments': False,
 'that': False,
 'are': False,
 'has': False,
 'for': False,
 'with': False,
 'lower': False,
 'an': False,
 'is': False,
 'by': False,
 'have': False,
 'will': False,
 'at': False,
 'high': False,
 'from': False,
 'company': False,
 'their': False,
 'or': False,
 'annual': False,
 'us': False,
 'which': False,
 'short': False,
 'minority': False,
 'foreign': False,
 'said': False,
 'higher': False,
 'as': False,
 'bonds': False,
 'income': False,
 'nt': False,
 'other': False,
 'it': False,
 'pay': False,
 'be': False,
 'below': False,
 'up': False,
 'would': False,
 'because': False,
 'short-term': False,
 'was': False,
 'buying': False,
 '$': False,
 'also': False,
 'pursue': False,
 'but': False,
 'best': False,
 'he': False,
 'million': False,
 'debt': False,
 'some': False,
 'expressed': Fa

##### 4.1 Prendre modèle sur https://www.nltk.org/book/ch06.html (début du 1.2)

##### 4.2 Pour le dictionnaire, il faut créer un trait pour chaque mot de vocabulary, et examiner si ce mot est présent dans une fenêtre de taille window_size autour de l’occurrence de interest : si oui, le trait est True, sinon il est False. Par exemple, on aboutit à : {'contains(the)': False, 'contains(,)': True, 'contains(rates)': True, …}.

##### 4.3 Ajouter aussi le trait ‘word0’ qui note si l’occurrence est interest ou interests (pluriel).

##### 4.3 Combien d’occurrences pour chaque sens de interest y a-t-il dans feature_sets ?

#### 5. Diviser les données de feature_sets en deux sous-ensembles : l’un comportant 80% des données est le train_set, et l’autre (20%) est le test_set. Attention, il faut respecter deux conditions :

##### 5.1 Chaque sens doit être présent dans les mêmes proportions dans train_set et dans test_set (donc il faut faire la division de manière séparée pour chaque sens).

##### 5.2 Mélanger avec shuffle() les occurrences avant de prendre les 80% premières pour le `train_set`et les 20% restantes dans le test_set.

#### 6. Entraîner un classifieur de type NaiveBayesClassifier de NLTK sur train_set, puis le tester sur les données de test_set. Quelle est la précision (accuracy) atteinte ?

#### 7. Adapter le code précédent pour effectuer plusieurs divisions des données en train et test (par exemple 10), et calculer la moyenne des scores obtenus. Comment se compare cette moyenne avec votre premier résultat ?

#### 8. Cherchez les meilleurs paramètres pour la taille de la fenêtre (p.ex. 1, 3, 5, 7, 11) et la taille du vocabulaire (50, 100, 200, 500, 1000 mots). Combien d’expériences faut-il exécuter ? Quelle est la meilleure combinaison fenêtre x vocabulaire et quel est le score moyen obtenu ?

### B. Traits lexicaux positionnels : valeurs des mots précédant/suivant interest
Pour cette deuxième partie, on réutilisera beaucoup d’éléments de la première. Seule la nature des
traits utilisés et leur extraction vont changer.

#### 1. Partir de la liste de listes de mots (une liste par phrase) précédente, appelée tokenized_sentences.

#### 2. Définir une variable window_size2, par exemple égale à 3 (on la fera varier plus tard).

#### 3. Parcourir les tokenized_sentences et pour chaque phrase créer un couple (dictionnaire, sens), où le dictionnaire regroupe les traits et leurs valeurs, et le sens est un nombre de 1 à 6 indiquant le sens de interest. Les couples pour toutes les phrases seront rassemblés dans une nouvelle liste appelée feature_sets2.

##### 3.1 Pour le dictionnaire de traits, il faut cette fois-ci créer un trait pour chaque position relative par rapport à interest, donc ‘word-1’, ‘word+1’, etc. (jusqu’à window_size2). La valeur du trait sera le mot trouvé à cette position, ou ‘NONE’ si on sort de la phrase. Par exemple {(‘word-1’ : ‘his’), (‘word+1’ : ‘in’), … }.

##### 3.2 Ajouter aussi le trait ‘word0’ qui note si l’occurrence est interest ou interests (pluriel).

#### 4. Diviser les données de feature_sets2 en deux sous-ensembles (80%/20%) appelés train_set2 et test_set2 avec la même procédure qu’à la partie A.

#### 5. Entraîner un classifieur de type NaiveBayesClassifier de NLTK sur train_set2, puis le tester sur les données de test_set2. Quelle est la précision (accuracy) atteinte ?

#### 6. Effectuer plusieurs divisions des données en train et test (par exemple 10), et calculer la moyenne des scores obtenus. Comment se compare cette moyenne avec votre premier résultat ?

#### 7. Cherchez les meilleurs paramètres pour la taille de la fenêtre (p.ex. entre 1 et 15). Quelle est la meilleure valeur et quel est le score moyen obtenu ?

#### 8. Quelle est le meilleur score obtenu entre (A) et (B) ?

#### 9. Thème de réflexion facultatif : les différences des scores sont-elles statistiquement significatives ?

Merci d’envoyer votre notebook Jupyter par email au professeur avant le **lundi 27 mai à 23h59**.